<a href="https://colab.research.google.com/github/aymericvie/Covid19_coevolution/blob/main/Covid_coevolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [270]:
# imports
import random
import numpy as np
random.seed(9)

# Parameters
virus_size = 5
initial_virus_population = 10
baseR = round(random.uniform(2,3),2)
mut = np.sort(np.random.normal(0,1,size=virus_size))
print(mut)

policy_size = 6
initial_policy_population = 4

initial_infected = 10

tmax = 7
crossover_rate_policy = 0.9
mutation_rate_virus = 0.1
mutation_rate_policy = 0.1

# Parameters for non pharmaceutical interventions (NPIs) from Haug et al (2020)
npi = [round(random.uniform(-0.36,-0.18),2), round(random.uniform(-0.24,-0.16),2), round(random.uniform(-0.24,-0.08),2), round(random.uniform(-0.18,-0.08),2), round(random.uniform(-0.18,-0.04),2), round(random.uniform(-0.18,0),2)]
#print(npi)

# Initial sampling for viruses
population_viruses = np.zeros((initial_virus_population,virus_size))
print("initial pop virus")
print(population_viruses)

# Initial sampling for policies
population_policies = np.zeros((initial_policy_population,policy_size))
#print(population_policies)

infected = initial_infected
population_virus_size = initial_virus_population
population_policy_size = initial_policy_population

###

# population_policies[0,0] = 1
# population_policies[0,1] = 1
# population_policies[0,2] = 1
# population_policies[1,1] = 1
# population_policies[2,2] = 1
# population_policies[3,3] = 1

print("initial pop polici")
print(population_policies)

# population_viruses[0,0] = 1
# population_viruses[1,1] = 1
# population_viruses[2,2] = 1
# population_viruses[3,3] = 1
# population_viruses[4,4] = 1

# determine efficacy of population viruses and policies

t = 0
# Initial results
distribution_virus_genome = np.mean(population_viruses,0)
distribution_policy_genome = np.mean(population_policies,0)
virusR_history = []
policyR_history = []


while t < tmax+1:

  virusR = np.zeros((population_virus_size,1))
  i = 0
  while i < population_virus_size:
    virusR[i,0] = baseR + np.sum(population_viruses, axis = 1)[i]
    i += 1
  print(virusR)


  policyR = np.zeros((population_policy_size,1))
  i = 0
  while i < population_policy_size:
    policyR[i,0] = np.dot(population_policies[i,],npi)
    i += 1

  #print(policyR)

  # evaluate coevolution fitness evaluation for policies, with tournament(3) selection
  policy_fitness = np.zeros((population_policy_size,1))

  i = 0
  while i < population_policy_size:
    a = random.randint(0,population_virus_size-1)
    b = random.randint(0,population_virus_size-1)
    c = random.randint(0,population_virus_size-1)
    policy_fitness[i,0] = (1/3)*(virusR[a,0] + virusR[b,0] + virusR[c,0]) + policyR[i,0]
    i += 1
  #print(policy_fitness)
  adjusted_policy_fitness = 1/(1+policy_fitness)
  #print(adjusted_policy_fitness)
  policy_selection = adjusted_policy_fitness / np.sum(adjusted_policy_fitness)
  #print(policy_selection)

  cumulative_policy_selection = np.copy(policy_selection)
  i = 1
  while i < len(cumulative_policy_selection):
    cumulative_policy_selection[i,0] += cumulative_policy_selection[i-1,0]
    i += 1

  i = 0
  while i < population_policy_size:
    a = random.uniform(0,1)
    b = 0
    while a > cumulative_policy_selection[b]:
        b += 1

    a = random.uniform(0,1)
    c = 0
    while a > cumulative_policy_selection[c]:
        c += 1

  # Do crossover between elements b and c

    offspring1 = np.zeros((1, policy_size))
    offspring2 = np.zeros((1, policy_size))

    n = random.uniform(0,1)
    if n <= crossover_rate_policy:

      cpoint = random.randint(0,policy_size)
      j = 0
      while j < cpoint:
        offspring1[0,j] = population_policies[b,j]
        offspring2[0,j] = population_policies[c,j]
        j += 1
      while j < policy_size:
        offspring1[0,j] = population_policies[c,j]
        offspring2[0,j] = population_policies[b,j]
        j += 1
      #print(offspring1)
      #print(offspring2)

    elif n > crossover_rate_policy:
      offspring1 = np.copy(population_policies[b,])
      offspring2 = np.copy(population_policies[c,])

    k = 0
    while k < policy_size:
      n = random.uniform(0,1)
      if n <= mutation_rate_policy:
        if offspring1[0,k] == 0:
          offspring1[0,k] = 1
        elif offspring1[0,k] == 1:
          offspring1[0,k] = 0
      k += 1

    k = 0
    while k < policy_size:
      n = random.uniform(0,1)
      if n <= mutation_rate_policy:
        if offspring2[0,k] == 0:
          offspring2[0,k] = 1
        elif offspring1[0,k] == 1:
          offspring2[0,k] = 0
      k += 1

    population_policies = np.vstack([population_policies,offspring1])
    population_policies = np.vstack([population_policies,offspring2])
    i += 2

  #print("new pop polici")
  population_policies = population_policies[population_policy_size:, :]
  #print(population_policies)


  # operate virus transmission and mutation
  #print(virusR + np.mean(policyR))
  i = 0
  while i < population_virus_size:
    j = 0
    while j < int(virusR[i,0]+np.mean(policyR)):
      candidate_virus = np.copy(population_viruses[i,])
      k = 0
      while k < virus_size:
        n = random.uniform(0,1)
        if n <= mutation_rate_virus:
          if candidate_virus[k] == 0:
            candidate_virus[k] = 1
          elif candidate_virus[k] == 1:
            candidate_virus[k] = 0
        k += 1
      #print(candidate_virus)
      population_viruses = np.vstack([population_viruses,candidate_virus])
      j += 1
    i += 1

  #print("new pop virus")
  population_viruses = population_viruses[population_virus_size:, :]
  #print(population_viruses)
  population_virus_size = population_viruses.shape[0]
  print(population_virus_size)

  # Save results
  if t > 0:
    distribution_virus_genome = np.vstack([distribution_virus_genome,np.mean(population_viruses,0)])
    distribution_policy_genome = np.vstack([distribution_policy_genome,np.mean(population_policies,0)])

  virusR_history.append(round(np.mean(virusR),2))
  policyR_history.append(round(np.mean(policyR),2))

  print("Current time: " + str(t)+"/"+str(tmax))
  t += 1



[-0.77039759 -0.01977248 -0.01739187  0.05424068  0.28210558]
initial pop virus
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
initial pop polici
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[2.46]
 [2.46]
 [2.46]
 [2.46]
 [2.46]
 [2.46]
 [2.46]
 [2.46]
 [2.46]
 [2.46]]
20
Current time: 0/7
[[5.46]
 [3.46]
 [3.46]
 [3.46]
 [2.46]
 [3.46]
 [3.46]
 [2.46]
 [3.46]
 [3.46]
 [3.46]
 [2.46]
 [2.46]
 [2.46]
 [3.46]
 [4.46]
 [2.46]
 [4.46]
 [2.46]
 [2.46]]
56
Current time: 1/7
[[5.46]
 [3.46]
 [5.46]
 [4.46]
 [6.46]
 [3.46]
 [4.46]
 [4.46]
 [5.46]
 [4.46]
 [3.46]
 [3.46]
 [4.46]
 [5.46]
 [2.46]
 [3.46]
 [4.46]
 [5.46]
 [4.46]
 [3.46]
 [4.46]
 [3.46]
 [3.46]
 [3.46]
 [3.46]
 [3.46]
 [3.46]
 [3.46]
 [2.46]
 [3.46]
 [2.46]
 [3.46]
 [3.46]
 [4.46]
 [2.46]
 [3.46]
 [2.46]
 [3.46]
 [3.46]
 [3.46]
 [4.46]
 [3.46]
 [4.46]
 [5.46]

In [271]:
print(distribution_virus_genome)
print(distribution_policy_genome)
print(virusR_history)
print(policyR_history)

[[0.         0.         0.         0.         0.        ]
 [0.21428571 0.375      0.30357143 0.21428571 0.28571429]
 [0.29473684 0.46315789 0.39473684 0.27368421 0.37894737]
 [0.39004149 0.52143845 0.46749654 0.35131397 0.4757953 ]
 [0.45511345 0.57448208 0.52449852 0.41269319 0.5251562 ]
 [0.50732064 0.6045388  0.56039531 0.46266471 0.55871157]
 [0.54575372 0.62381658 0.5896748  0.50549793 0.59117211]
 [0.57650713 0.63602142 0.60919839 0.54193687 0.61098745]]
[[0.   0.   0.   0.   0.   0.  ]
 [0.   0.5  0.   0.5  0.5  0.5 ]
 [0.25 0.5  0.   0.5  0.   0.25]
 [0.25 0.75 0.   0.25 0.   0.25]
 [0.25 0.5  0.   0.   0.   0.25]
 [0.25 0.5  0.25 0.   0.   0.25]
 [0.25 0.75 0.25 0.   0.   0.25]
 [0.   0.75 0.25 0.   0.   0.75]]
[2.46, 3.26, 3.85, 4.27, 4.67, 4.95, 5.15, 5.32]
[0.0, -0.16, -0.27, -0.28, -0.3, -0.19, -0.22, -0.28]


In [259]:
print(mut)

[-0.80221523  0.09783349  0.2093148   0.59814134  1.43166322]
